In [6]:
%pip install pandas
%pip install matplotlib
%pip install pyarrow
%pip install dask
%pip install toolkit

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip in

In [16]:
%pip install pipreqs
%pip install --use-deprecated=legacy-resolver -r /Users/ariel/Documents/hackthon-petro/toolkit/requirements.txt

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
  Using cached lightgbm-4.5.0-py3-none-macosx_12_0_arm64.whl (1.6 MB)
  Using cached plotly-5.24.1-py3-none-any.whl (19.1 MB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
     |████████████████████████████████| 54 kB 2.7 MB/s eta 0:00:011
     |████████████████████████████████| 374 kB 994 kB/s eta 0:00:01
     |████████████████████████████████| 2.7 MB 8.8 MB/s eta 0:00:01
     |████████████████████████████████| 28.8 MB 19.1 MB/s eta 0:00:01
  Attempting uninstall: seaborn
    Found existing installation: seaborn 0.12.0
    Uninstalling seaborn-0.12.0:
      Successfully 

In [22]:
import os
from pathlib import Path
from toolkit.misc import load_instance, plot_instance

# Caminho base onde estão os subdiretórios do dataset
base_path = '/Users/ariel/Documents/hackthon-petro/dataset/'

# Função para percorrer subdiretórios e coletar instâncias
def carregar_instancias_com_subdiretorios(base_path):
    instances = []
    
    # Percorrer todos os subdiretórios
    for root, dirs, files in os.walk(base_path):
        for file_name in files:
            if file_name.endswith('.parquet'):  # Ajuste a extensão conforme necessário
                file_path = Path(os.path.join(root, file_name))  # Converter para Path
                label = int(os.path.basename(root))  # Supondo que o rótulo seja o nome da pasta
                instances.append((label, file_path))  # Adicionar a tupla (rótulo, caminho)
    
    return instances

# Carregar as instâncias de todos os subdiretórios
instances = carregar_instancias_com_subdiretorios(base_path)

# Exibir o número de instâncias carregadas
print(f"Número de instâncias carregadas: {len(instances)}")

# Usar a função load_instance para processar cada instância (exemplo)
for label, instance_path in instances:
    instance_data = load_instance((label, instance_path))  # Passar como tupla (label, file_path)
    if instance_data is not None:
        print(f"Instância carregada com sucesso: {instance_path}")
        
        # Exibir a primeira instância
        plot_instance(class_number=label, instance_index=0, resample_factor=100)  # Ajuste conforme necessário
        break  # Visualizar apenas a primeira instância


Número de instâncias carregadas: 2228
Instância carregada com sucesso: /Users/ariel/Documents/hackthon-petro/dataset/9/SIMULATED_00098.parquet


In [6]:
import pandas as pd

# Função para corrigir o DataFrame após pivot
def corrigir_pivot(df_instances):
    try:
        # Tentativa de agrupar e pivotar conforme a forma correta
        toi = df_instances.pivot(index="INSTANCE LABEL", columns="SOURCE", values=0)  # Corrigir o pivot aqui
        toi = toi.fillna(0).astype(int).T
        return toi
    except Exception as e:
        print(f"Erro capturado no pivot: {e}")
        return None

# Função principal para carregar e explorar eventos
def explorar_eventos_e_instancias(base_path):
    # 1. Exibir todos os nomes de eventos no dataset
    print("Eventos no Dataset:", EVENT_NAMES)
    
    # 2. Escolher o primeiro evento para explorar suas propriedades
    evento_escolhido = EVENT_NAMES[0]
    explorar_evento(evento_escolhido)

    # 3. Carregar as instâncias de todos os subdiretórios
    instances = carregar_instancias_com_subdiretorios(base_path)
    print(f"Número de instâncias carregadas: {len(instances)}")
    
    # 4. Processar a primeira instância com load_instance e plot_instance
    for label, instance_path in instances:
        instance_data = load_instance((label, instance_path))  # Passar como tupla (label, file_path)
        if instance_data is not None:
            print(f"Instância carregada com sucesso: {instance_path}")
            # Exibir a primeira instância
            plot_instance(class_number=label, instance_index=0, resample_factor=100)  # Ajuste conforme necessário
            break  # Visualizar apenas a primeira instância

    # 5. Gerar a tabela de instâncias (reais, simuladas, desenhadas à mão)
    real_instances, simulated_instances, drawn_instances = get_all_labels_and_files()

    # Supondo que create_table_of_instances devolve df_instances
    df_instances = create_table_of_instances(real_instances, simulated_instances, drawn_instances)

    # Aplicar a correção manual no pivot
    table_of_instances_corrigida = corrigir_pivot(df_instances)

    # Exibir a tabela corrigida
    if table_of_instances_corrigida is not None:
        print("\nTabela de Instâncias Corrigida:")
        print(table_of_instances_corrigida)

        # 6. Filtrar eventos raros (com limite de 5%) usando a tabela corrigida
        eventos_raros = filter_rare_undesirable_events(table_of_instances_corrigida, threshold=0.05)
        print("\nEventos raros:")
        print(eventos_raros)
    else:
        print("Erro ao corrigir a tabela de instâncias.")

# Definir o caminho base onde os arquivos estão armazenados
base_path = '/Users/ariel/Documents/hackthon-petro/dataset/'

# Executar a função principal
explorar_eventos_e_instancias(base_path)


Eventos no Dataset: ['NORMAL', 'ABRUPT_INCREASE_OF_BSW', 'SPURIOUS_CLOSURE_OF_DHSV', 'SEVERE_SLUGGING', 'FLOW_INSTABILITY', 'RAPID_PRODUCTIVITY_LOSS', 'QUICK_RESTRICTION_IN_PCK', 'SCALING_IN_PCK', 'HYDRATE_IN_PRODUCTION_LINE', 'HYDRATE_IN_SERVICE_LINE']
Evento: NORMAL
Descrição: Normal Operation
Etiqueta (Label): 0
É Transiente: None
Tamanho da Janela: None
Passo: None

Número de instâncias carregadas: 2228
Instância carregada com sucesso: /Users/ariel/Documents/hackthon-petro/dataset/9/SIMULATED_00098.parquet


TypeError: pivot() takes 1 positional argument but 4 were given